In [1]:
# 以下代码的目的在于解决【中文字符】问题
import sys
stdout = sys.stdout
# 新版sys, 需要先reload(sys)再进行sys.setdefaultencoding操作
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [29]:
import pandas as pd 
import numpy as np
import os
# 设置工作目录
os.chdir('/Users/Max/PycharmProjects/user_churn_second')

# visual
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

# import basic 
import numpy as np

# machine learning 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.utils import resample
from sklearn.preprocessing import add_dummy_feature
from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation
from collections import Counter

# import third-party
import pandas as pd 
from collections import Counter

# import local file

## 工作流程

1、读取已有的X特征集

2、对X进行预处理

3、异常值剔除

4、得到dummy变量

5、拆分测试数据集与训练数据集

6、归一化

7、提取重要特征

8、训练模型

9、保存模型

### 1、读取特征值

In [153]:
X = pd.read_csv("output/extract_result.csv", index_col=0).drop(['actions', 'date', 'path3', 'path4', 'path5', 'path6'], axis=1).drop_duplicates(keep='first')
y = pd.read_csv('output/new_3days_y_2018-03-24.csv', index_col=0).drop_duplicates(keep='first')

### 2、数据预处理部分

In [154]:
import data_clean as dc 
from imp import reload
reload(dc)
dc_dc = dc.data_clean()
X_y = pd.merge(X, y, left_on='userid', right_on='full_id', how='left')
X_y = dc_dc.quick_correct(X_y)

### 3、异常值处理

In [155]:
# 异常值处理
# kick outlier out
# 1. outlier 
X_y = dc_dc.outlier_deal(X_y)

### 4、数据转化

In [156]:
# 得到 dummies 变量
y_fea = X_y.loc[:, 'y']
X_fea = pd.get_dummies(X_y.drop(['y', 'userid', 'full_id'], axis=1))

### 5、拆分训练&测试数据集

In [157]:
# train & test 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_fea, y_fea, test_size=0.3, random_state=2)

### 6、归一化处理

In [158]:
# scale 
# 归一化
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaler = scaler.transform(X_train)
X_test_scaler = scaler.transform(X_test)
# assert
assert X_test_scaler.max() != 0, ('right')
assert X_train_scaler.max() == 1, ('right')
print np.shape(X_train)
print np.shape(X_test)
print np.shape(np.vstack((X_train, X_test)))

(47481, 34)
(20350, 34)
(67831, 34)


### 7、提取重要特征

In [159]:
# train
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
X, y = X_train, y_train
X.shape

clf_fea = ExtraTreesClassifier()
clf_fea_fit = clf_fea.fit(X, y)
clf_fea_fit.feature_importances_  

model = SelectFromModel(clf_fea, prefit=True)
X_new = model.transform(X)
X_new.shape     

(47481, 7)

In [160]:
clf_features = zip(X_fea.columns, clf_fea.feature_importances_)
clf_features_sort = sorted(clf_features, key=lambda x: x[1], reverse=True)
clf_features_sort

[('read', 0.20576607761404073),
 ('today_read_rate', 0.15321825219681842),
 ('path1', 0.12361312187667886),
 ('favorite', 0.093843027639713514),
 ('sub', 0.061463210152518391),
 ('purchase', 0.045823417858793368),
 ('share', 0.031068014490180495),
 ('weekend', 0.028412814825854456),
 ('unsub', 0.026504115817554574),
 ('SignIn', 0.021399612078795614),
 ('os_Android', 0.018020270062236873),
 ('path2', 0.013851797276811292),
 ('lucky_money', 0.013205088085282795),
 ('status_student', 0.01191311706834779),
 ('status_worker', 0.01141683012054117),
 ('gender_female', 0.010999426195619168),
 ('city_first', 0.010884763488296124),
 ('os_missing', 0.010146024362310482),
 ('gender_male', 0.009798505516090239),
 ('city_second', 0.0097499419950013485),
 ('fromchannel_badchannel', 0.0089577148433093914),
 ('city_others', 0.0083673893206318746),
 ('coin', 0.0081179018206395236),
 ('os_iOS', 0.0078811054107038522),
 ('fromchannel_Oppo', 0.0073517912453741298),
 ('fromchannel_Huawei', 0.007060463104446

### 8、训练模型

In [37]:
# input: X_train_f, X_test_f, y_train_f, y_test_f
# outputs: top5 favorited buyers for each user, and sorted by pred_score
# after that: for rest of buyers, give possible love score

# Train model
clf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, min_samples_split=5, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_split=0.0, bootstrap=True, oob_score=True, n_jobs=-1, random_state=21, verbose=0, warm_start=False, class_weight='balanced')

# set parameters
parameters = dict(
n_estimators = [95, 100, 105], 
min_samples_split = [15, 20, 25, 30],
min_samples_leaf = [5, 10], 
class_weight = [None]
)

# set score function
def my_custom_scorer(y_true, y_pred):
    return recall_score(y_true, y_pred, average='micro')
scoring = make_scorer(my_custom_scorer)
# set 
estimator = GridSearchCV(clf,parameters, cv = 10, n_jobs=-1, scoring=scoring)

# train
print "训练模型中...."
estimator.fit(X_train_scaler, y_train)
print "训练完成...."

print "预测中...."
# predict on test
clf_pred_y = estimator.predict(X_test_scaler)
 
# Is our model still predicting just one class?
# check
print( np.unique( clf_pred_y ))
 
# How's our accuracy?
#print(accuracy_score(y_test, clf_pred_y))
# 0.9744
 
# What about AUROC?
#prob_y = clf.predict_proba(X_test_scaler)
#prob_y = [p[1] for p in prob_y]
# best params
print "best estimator_rf\n{}\n{}\n\n".format(estimator.best_estimator_, estimator.best_params_)

# analysis report
print "分析报告生成中....."
print(classification_report(y_train, estimator.predict(X_train_scaler)))
print(classification_report(y_test, clf_pred_y))
print (classification_report(np.concatenate((y_train, y_test), axis=0), estimator.predict(np.vstack((X_train_scaler, X_test_scaler))))) # favorite 2, recall is not good

训练模型中....
训练完成....
预测中....
[0 1]
best estimator_rf
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=0.0, min_samples_leaf=5,
            min_samples_split=25, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=True, random_state=21,
            verbose=0, warm_start=False)
{'min_samples_split': 25, 'n_estimators': 100, 'min_samples_leaf': 5, 'class_weight': None}


分析报告生成中.....
             precision    recall  f1-score   support

          0       0.80      0.97      0.88     35372
          1       0.79      0.29      0.42     12109

avg / total       0.80      0.80      0.76     47481

             precision    recall  f1-score   support

          0       0.78      0.96      0.86     15124
          1       0.66      0.23      0.34      5226

avg / total       0.75      0.77      0.73     20350

             precision    recal

### 9、保存模型

In [47]:
# 保存模型到output
joblib.dump(estimator, "output/train_model.m")

['output/train_model.m']

In [50]:
# 恢复模型并做预测
from sklearn.externals import joblib
recoverd_model = joblib.load("output/train_model.m")
recoverd_model.score(X_test_scaler, y_test)

0.77100737100737105

In [162]:
X_fea.columns

Index([u'completed_info', u'SignIn', u'weekend', u'coin', u'lucky_money',
       u'unsub', u'sub', u'purchase', u'share', u'favorite', u'read', u'path1',
       u'path2', u'today_read_rate', u'os_Android', u'os_iOS', u'os_missing',
       u'city_first', u'city_others', u'city_second', u'gender_female',
       u'gender_male', u'gender_missing', u'status_missing', u'status_other',
       u'status_student', u'status_worker', u'fromchannel_Huawei',
       u'fromchannel_Oppo', u'fromchannel_Tencent', u'fromchannel_Vivo',
       u'fromchannel_Xiaomi', u'fromchannel_badchannel',
       u'fromchannel_unknown_ios'],
      dtype='object')

In [163]:
model_columns = np.loadtxt('output/model_columns', dtype='S')


In [164]:
model_columns

array(['completed_info', 'SignIn', 'weekend', 'coin', 'lucky_money',
       'unsub', 'sub', 'purchase', 'share', 'favorite', 'read', 'path1',
       'path2', 'today_read_rate', 'os_Android', 'os_iOS', 'os_missing',
       'city_first', 'city_others', 'city_second', 'gender_female',
       'gender_male', 'gender_missing', 'status_missing', 'status_other',
       'status_student', 'status_worker', 'fromchannel_Huawei',
       'fromchannel_Oppo', 'fromchannel_Tencent', 'fromchannel_Vivo',
       'fromchannel_Xiaomi', 'fromchannel_badchannel',
       'fromchannel_unknown_ios'], 
      dtype='|S23')